#### Concepts

Class has attributes which are fields and methods.
Fields include instance variables(self.v) and class variables(cls.v).
Methods include static methods, class methods(cls) and normal methods(self).

In [6]:
class User:
    count = 0
    __privilege = 'class 1'
    
    def __init__(self, username):
        self.username = username
        self.__passwd = '123456'
        User.count += 1
        
    @property
    def intro(self):
        self.intro = 'i\'m {self.username}'.format(self)
        return self.intro
    @intro.setter
    def intro(self, value):
        self.username = value
    @intro.deleter
    def intro(self):
        del self.intro
        
    @classmethod
    def isClassOf(cls, base):
        return type(cls) == base
    
    @staticmethod
    def countAll():
        print('total num of users is {}'.format(User.count))
        
        
user = User('test')
user.countAll()
        

total num of users is 1


#### Name Mangling

In [6]:
class Person:
    __number = 0
    def __init__(self, name):
        self.name = name
        self.age = Person.__number # self.__number
    def introduce(self):
        print('The person is '+str(self.name)+' and is '+str(self.age))
class Student(Person):
    __number = 10
    school = 'Omega'
    def __init__(self, name, age, grade):
        # Person.__init__(self)
        super(Student, self).__init__(name, age)
        self.grade = grade
    def introduce(self):
        print('The student is '+str(self.name)+' and is '+str(self.age)+' in grade '+str(self.grade))
print(Student._Student__number)
print(Student._Person__number)
person = Person('jeff')
print(person._Person__number)

10
0
0


#### Inheritance

In [11]:
class Base(object):
    def __init__(self):
        print("enter Base")
        print("leave Base")
 
class A(Base):
    def __init__(self):
        print("enter A")
        super(A, self).__init__()
        print("leave A")
 
class B(Base):
    def __init__(self):
        print("enter B")
        super(B, self).__init__()
        print("leave B")
 
class C(A, B):
    def __init__(self):
        print("enter C")
        super(C, self).__init__()
        print("leave C")
        
c = C()

enter C
enter A
enter B
enter Base
leave Base
leave B
leave A
leave C


#### Special Methods

__init__, __new__
__str__, __repr__
__getitem__、__setitem__、__delitem__
__dir__
__call__
__metaclass__

__new__ vs __init__

__new__ is a static method, while __init__ is a instance method.
__new__ returns an instance of class, __init__ does not.
__init__ is invoked after __new__ returns an instance of cls.
__new__ is for creating an instance whereas __init__ is for initiating the instance.

In [ ]:
class Mapping:
    def __init__(self, iterable):
        self.items_list = []
        self.__update(iterable)

    def update(self, iterable):
        for item in iterable:
            self.items_list.append(item)

    __update = update   # private copy of original update() method

class MappingSubclass(Mapping):

    def update(self, keys, values):
        # provides new signature for update()
        # but does not break __init__()
        for item in zip(keys, values):
            self.items_list.append(item)

dir() vs var()

Without arguments, return the list of names in the current local scope. With an argument, attempt to return a list of valid attributes for that object.

The default dir() mechanism behaves differently with different types of objects, as it attempts to produce the most relevant, rather than complete, information:

If the object is a module object, the list contains the names of the module’s attributes.
If the object is a type or class object, the list contains the names of its attributes, and recursively of the attributes of its bases.
Otherwise, the list contains the object’s attributes’ names, the names of its class’s attributes, and recursively of the attributes of its class’s base classes.

So the difference between vars(x) and dir(x) is that dir(x) does the extra work of looking in x's class (and its bases) for attributes that are accessible from it, not just those attributes that are stored in x's own symbol table.

In [8]:
p = Person('tom')
print(dir(Student))
print(vars(Student))

['_Person__number', '_Student__number', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'introduce', 'school']
{'__module__': '__main__', '__doc__': None, '_Student__number': 10, 'school': 'Omega', 'introduce': <function Student.introduce at 0x105a56048>, '__init__': <function Student.__init__ at 0x105a56158>}


__slots__

In [14]:
class Person:
    number = 0
    __slots__ = ['name', 'age']
    def __init__(self, name, age):
        self.name = name
        self.age = age

p = Person('linoel', 12)
p.sex = 'male' # cannot add because of __slots__
del p.number # instance cannot delete class's attribute

AttributeError: 'Person' object attribute 'number' is read-only

In [36]:
class MyDescriptor:
    def __init__(self, name):
        self.name = name
    def __get__(self, obj, objtype):
        print('getting from descriptor...')
        return self.name
    def __set__(self, obj, value):
        print('setting via descriptor...')
        self.name = value
class Person:
    number = MyDescriptor(0)
    def __init__(self, name, age, number):
        self.name = name
        self.age = age
        self.number = number # attribute assigning via MyDescriptor
        # self.__dict__['number'] = number # result is 0 instead of 1, prove that data descriptor prior to instance dict
        print('init')
    def __len__(self):
        return 10
    def __getattribute__(self, name): # will only be called for instance of class, not the class
        print('invoke __getattribute__')
        return super(Person, self).__getattribute__(name)
        #return object.__getattribute__(self, name) another way to do it
    def __introduce(self):
        print('The person is '+str(self.name)+' and is '+str(self.age))
p = Person('m', 11, 1)
p.number # attribute access will start from instance's dict, then its class or parent class's dict
# Person.number == Person.__dict__['number'].__get__(None, Person) # True, both will call MyDescriptor, but not __getattribute__
#print(Person.__len__) # will not call __getattribute__, will call __getattribute__ in its metaclass
#print(p.__len__) # will call __getattribute__

setting via descriptor...
init
invoke __getattribute__
getting from descriptor...


1

#### Enum

In [32]:
from enum import Enum

# usage 1
Week = Enum('Week', ('Mon', 'Tue', 'Wed'))
# print(Week.__members__) # OrderedDict([('Mon', <Week.Mon: 1>), ('Tue', <Week.Tue: 2>), ('Wed', <Week.Wed: 3>)])
for name, member in Week.__members__.items():
    print(name)
    print(type(member)) # <enum 'Week'>
    print('{} : {}'.format(member.name, member.value))
    
# usage 2
class Week(Enum):
    Mon = 'Monday'
    Tue = 'Tuesday'
    Wed = 'Wednesday'

# get enum member
print(Week['Mon'])
print(Week('Monday'))
print(Week.Mon)

Mon
<enum 'Week'>
Mon : 1
Tue
<enum 'Week'>
Tue : 2
Wed
<enum 'Week'>
Wed : 3
Week.Mon
Week.Mon
Week.Mon


#### metaclass

type can be used to create class.
In fact, class keyword invokes type to create class.

In [33]:
def func(self, str='world'):
    print('hello %s' % str)

Hello = type('Hello', (object,), dict(hello=func))
h = Hello()
h.hello()

hello world


use metaclass to define class creation.
class can be seen as instance of metaclass.

In [34]:
class ListMetaclass(type):
    def __new__(cls, name, bases, attrs): # here cls refers to ListMetaclass
        attrs['add'] = lambda self, value: self.append(value)
        return type.__new__(cls, name, bases, attrs)

class MyList(list, metaclass=ListMetaclass):
    pass

l = MyList()
l.add(1)
print(l)

[1]


#### type vs object

object is as the base class of all classes, including type.
type defines the type of all classes, including itself.

In [35]:
print(object.__bases__)
print(object.__class__)
print(type.__bases__)
print(type.__class__)

()
<class 'type'>
(<class 'object'>,)
<class 'type'>
